In [15]:
import os
import sys
import pandas as pd
import numpy as np
import pyarrow
#import re
from email.parser import Parser
import time
from datetime import datetime

In [16]:
# path = sys.argv[1]
path = "/Users/fanli/data/maildir"

In [17]:
def user(addr):
    if '@enron.com' not in addr:
        return None
    addr = addr[0:addr.index('@')]
    if '<' in addr or '#' in addr or "/o" in addr:
        return None
    if "'" in addr:
        addr = addr.replace("'", "")
    if len(addr)>0 and addr[0] == '.':
        addr = addr[1:]
    if len(addr)==0:
        return None
    return addr

In [43]:
def enrondf(path):
    # timer start
    start = time.time()
    
    filename = []
    Date = []
    From =[]
    To =[]
    Date = []
    Subject = []
    MailID = []
    Recipients = []

    mailid = 0
    for root, dircs, files in sorted(os.walk(path)):
        for name in sorted(files):
            if name == ".DS_Store":
                continue
            ### create fullpath
            fullpath = os.path.join(root,name)

            ### filter To
            with open(fullpath, encoding='latin1') as f:
                content = f.read()
                email = Parser().parsestr(content, headersonly=True) 
                #email = Parser().parse(content,headersonly=True)

                ### try out user function
                if email['To'] and email['From']:
                    normal_receiver = list(filter(None,[user(i.strip()) for i in email["To"].split(',')]))
                    normal_sender = user(email["From"])

                    if normal_receiver and normal_sender:
                        mailid+=1
                        for i in normal_receiver:
                            To.append(i)
                            From.append(normal_sender)
                            filename.append(fullpath.split('maildir/')[1])
                            
                            # replace invalid dates
                            date = email["Date"].split(":")[0][:-3].split(", ")[1]
                            if date.split(' ')[2] == '0001':
                                date = date.replace('0001','2001')
                            elif date.split(' ')[2] == '0002':
                                date = date.replace('0002','2002')
                            # append date
                            Date.append(date)
                            
                            # subjects
                            Subject.append(content.split("Subject: ")[1].split("\n")[0])
                            #print(email["Subject"].split("\n")[0],"\n++++++++++")
                            
                            # append mail ids
                            MailID.append(mailid)
                            # append number of Recipients
                            Recipients.append(len(normal_receiver))
                        
    Subject[0] += ' '
    Subject[3] += '  '
    Subject[6] += '   '
    Subject[7] += '    '
    
    df_dict = {'MailID':MailID,
                'Date': Date,
                'From': From,
                'To': To,
                'Recipients':Recipients,
                'Subject': Subject,
                'filename':filename}

    enrondf = pd.DataFrame.from_dict(df_dict)

    # change dtype to datetime
    enrondf['Date'] = pd.to_datetime(enrondf['Date'],format="%d %b %Y", errors = 'coerce').dt.strftime("%Y-%m-%d")
    
    # timer done
    done = time.time()
    # duration
    print((done-start)/60)
    
    return enrondf, Subject

In [41]:
df, sub= enrondf(path)
len(df['Subject'].unique())

2.4804363171259562


104834

In [42]:
df.head(10)

,MailID,Date,From,To,Recipients,Subject,filename
0,1,2001-05-14,phillip.allen,tim.belden,1,,allen-p/_sent_mail/1.
1,2,2001-05-04,phillip.allen,john.lavorato,1,Re:,allen-p/_sent_mail/10.
2,3,2000-10-18,phillip.allen,leah.arsdall,1,Re: test,allen-p/_sent_mail/100.
3,4,2000-10-23,phillip.allen,randall.gay,1,,allen-p/_sent_mail/1000.
4,5,2000-08-31,phillip.allen,greg.piper,1,Re: Hello,allen-p/_sent_mail/1001.
5,6,2000-08-31,phillip.allen,greg.piper,1,Re: Hello,allen-p/_sent_mail/1002.
6,7,2000-08-22,phillip.allen,david.l.johnson,2,,allen-p/_sent_mail/1003.
7,7,2000-08-22,phillip.allen,john.shafer,2,,allen-p/_sent_mail/1003.
8,8,2000-07-14,phillip.allen,joyce.teixeira,1,Re: PRC review - phone calls,allen-p/_sent_mail/1004.
9,9,2000-10-17,phillip.allen,mark.scott,1,Re: High Speed Internet Access,allen-p/_sent_mail/101.


# debug # of unique subjects (expect 104834)

# save to_feather

In [7]:
df.to_feather('./enron.feather')